<h1>Projeto De Previsao de Compras</h1>
<h2>Versao.: 1.2</h2>
<h2>Data.:   05/02/2019  08:49</h2>

<h3>
    TODO:<br/>
    <s>1º Reajustar a média móvel;</s><br/>
    <s>2º Calibrar com o auto-arima os modelos arima e sarima;</s><br/>
        3º Gerar o erro quadrático para cada modelo;<br/>
        4º Fazer a seleção das quantidades a serem compradas do modelo que contiver o menor erro;<br/>
        5º Rodar o projeto com todos os itens.<br/>
</h3>

In [204]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
from math import sqrt
from math import floor
from math import ceil
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

%matplotlib inline

try:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Users\PC\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], 
                        index_col='dt_venda', date_parser=dateparse)
except:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Users\PC\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], 
                       index_col='dt_venda', date_parser=dateparse)
    


DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
dataFrame.head()

,cd_empresa,cd_mercadoria,qt_vendida
dt_venda,,,
2016-01-01,1,117661,616
2016-02-01,1,117661,207
2016-03-01,1,117661,584
2016-04-01,1,117661,236
2016-05-01,1,117661,1002


In [3]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

In [4]:
def adicionarIndicesFaltantes(df):
    idx = pd.date_range(min(df.index), dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

In [181]:
def suavizacaoExponencialSimples(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste)) 
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [182]:
def holtLinear(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [183]:
def holtWinter(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    

In [213]:
def arima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    modelo_auto = auto_arima(ts, m = 12, seasonal = True, trace = False)
    
    arimaOrder = modelo_auto.order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    
    

In [214]:
def sarima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1]
    
    modelo_auto = auto_arima(ts, m = 12, seasonal = True, trace = False)
    
    sarimaOrder = modelo_auto.order
    seasonalOrder = modelo_auto.seasonal_order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    return dataFrameMercadoria
    

In [198]:
def calcularMediaMovel(dataFrameMercadoria):
    j = 1
    while j <= 12:
        
        mediasMoveisCalculada = dataFrameMercadoria['qt_vendida'].rolling(j).mean()
        
        data_minima = mediasMoveisCalculada.index.min()+1
        data_maxima = mediasMoveisCalculada.index.max()
        
        date_index = pd.date_range(data_minima, data_maxima, freq="MS")
        serie = pd.Series(index=date_index)
        
        for i, iValue in serie.iteritems():
            serie[i] = mediasMoveisCalculada[i-1]
          
        dataFrameMercadoria['MM'+str(j)] = serie;

        j += 1
    return dataFrameMercadoria

In [231]:
def calcularErro(dataFrameMercadoria, dataAtual):
    for i, iValue in dataFrameMercadoria.iterrows():
        if (len(iValue) > 7):            
            print(iValue)
            #aVendida = iValue.iloc[-6:, 3].values            
            #j = 1
            #while j <= 12:
                #aMM = df.iloc[-8:-2, j+2].values
                #erroMM = sqrt(mean_squared_error(aVendida, aMM))
                #df.loc[dataAtual, 'MM'+str(j)] = erroMM
                #j+=1

In [232]:
#arrayMercadoria = dataFrame['cd_mercadoria'].unique()
#arrayEmpresa = [1, 4, 5]
arrayMercadoria = [117661]
arrayEmpresa = [1]

arrayDeDataFrame = []

dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]
for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'],ascending=True)
        if (len(df) > 0):
            df = adicionarIndicesFaltantes(df)
            
            df = calcularMediaMovel(df)
            
            #df = suavizacaoExponencialSimples(df)
            
            #df = holtLinear(df)
            
            #df = holtWinter(df)
        
            #df = arima(df)
            
            #df = sarima(df)
            
            calcularErro(df, dataAtual)
            
            arrayDeDataFrame.append(df)
            
        #arrayDeDataFrame[len(arrayDeDataFrame)-1].fillna(0, inplace=True)
        
#calcularErro(dataAtual)
         

cd_empresa            1.0
cd_mercadoria    117661.0
qt_vendida          616.0
MM1                   NaN
MM2                   NaN
MM3                   NaN
MM4                   NaN
MM5                   NaN
MM6                   NaN
MM7                   NaN
MM8                   NaN
MM9                   NaN
MM10                  NaN
MM11                  NaN
MM12                  NaN
Name: 2016-01-01 00:00:00, dtype: float64
cd_empresa            1.0
cd_mercadoria    117661.0
qt_vendida          207.0
MM1                 616.0
MM2                   NaN
MM3                   NaN
MM4                   NaN
MM5                   NaN
MM6                   NaN
MM7                   NaN
MM8                   NaN
MM9                   NaN
MM10                  NaN
MM11                  NaN
MM12                  NaN
Name: 2016-02-01 00:00:00, dtype: float64
cd_empresa            1.0
cd_mercadoria    117661.0
qt_vendida          584.0
MM1                 207.0
MM2                 411.5
MM3   

Name: 2017-05-01 00:00:00, dtype: float64
cd_empresa            1.000000
cd_mercadoria    117661.000000
qt_vendida          632.000000
MM1                 645.000000
MM2                 572.500000
MM3                 671.000000
MM4                 626.750000
MM5                 659.000000
MM6                 668.000000
MM7                 702.000000
MM8                 614.250000
MM9                 546.000000
MM10                543.800000
MM11                582.909091
MM12                556.083333
Name: 2017-06-01 00:00:00, dtype: float64
cd_empresa            1.000000
cd_mercadoria    117661.000000
qt_vendida          365.000000
MM1                 632.000000
MM2                 638.500000
MM3                 592.333333
MM4                 661.250000
MM5                 627.800000
MM6                 654.500000
MM7                 662.857143
MM8                 693.250000
MM9                 616.222222
MM10                554.600000
MM11                551.818182
MM12             

Name: 2018-10-01 00:00:00, dtype: float64
cd_empresa            1.000000
cd_mercadoria    117661.000000
qt_vendida          250.000000
MM1                 322.000000
MM2                 382.000000
MM3                 385.666667
MM4                 398.500000
MM5                 403.200000
MM6                 384.666667
MM7                 401.571429
MM8                 388.750000
MM9                 406.444444
MM10                426.700000
MM11                402.909091
MM12                476.750000
Name: 2018-11-01 00:00:00, dtype: float64
cd_empresa            1.000000
cd_mercadoria    117661.000000
qt_vendida          488.000000
MM1                 250.000000
MM2                 286.000000
MM3                 338.000000
MM4                 351.750000
MM5                 368.800000
MM6                 377.666667
MM7                 365.428571
MM8                 382.625000
MM9                 373.333333
MM10                390.800000
MM11                410.636364
MM12             

In [225]:
arrayDeDataFrame[0]

,cd_empresa,cd_mercadoria,qt_vendida,MM1,MM2,MM3,MM4,MM5,MM6,MM7,MM8,MM9,MM10,MM11,MM12
2016-01-01,1,117661,616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-01,1,117661,207,616.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01,1,117661,584,207.0,411.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-01,1,117661,236,584.0,395.5,469.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-01,1,117661,1002,236.0,410.0,342.333333,410.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01,1,117661,261,1002.0,619.0,607.333333,507.25,529.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-01,1,117661,974,261.0,631.5,499.666667,520.75,458.0,484.333333,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-01,1,117661,524,974.0,617.5,745.666667,618.25,611.4,544.000000,554.285714,NaN,NaN,NaN,NaN,NaN
2016-09-01,0,0,0,524.0,749.0,586.333333,690.25,599.4,596.833333,541.142857,550.500,NaN,NaN,NaN,NaN
2016-10-01,0,0,0,0.0,262.0,499.333333,439.75,552.2,499.500000,511.571429,473.500,489.333333,NaN,NaN,NaN
